In [1]:
# Reading the text dataset
with open('dataset.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print("Length of dataset in chars: ", len(text))

Length of dataset in chars:  1115394


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab_size

65

In [4]:
# create a mapping from characters to integers
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}

# encoder: take a string and output a list of integers
encode = lambda s: [stoi[c] for c in s]

# decoder: take a list of integers and output a string
decode = lambda l: ''.join([itos[e] for e in l])

decode(encode("hey there")) == "hey there"

True

In [5]:
import torch

In [6]:
data = torch.tensor(encode(text))
data.shape, data.dtype

(torch.Size([1115394]), torch.int64)

In [7]:
# split the train and val data
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
print("Train and val data shapes: ", train_data.shape, val_data.shape)

Train and val data shapes:  torch.Size([1003854]) torch.Size([111540])


In [8]:
# context length/block size is chunks of the data sampled randomly for training
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1] # context is chars upto t including t
    target = y[t]
    print(f"when the input is {context}, the target is {target}")

when the input is tensor([18]), the target is 47
when the input is tensor([18, 47]), the target is 56
when the input is tensor([18, 47, 56]), the target is 57
when the input is tensor([18, 47, 56, 57]), the target is 58
when the input is tensor([18, 47, 56, 57, 58]), the target is 1
when the input is tensor([18, 47, 56, 57, 58,  1]), the target is 15
when the input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47
when the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58


In [10]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
xb.shape, yb.shape

(torch.Size([4, 8]), torch.Size([4, 8]))

In [11]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1] # context is chars upto t including t
        target = yb[b, t]
        print(f"when the input is {context}, the target is {target}")

when the input is tensor([24]), the target is 43
when the input is tensor([24, 43]), the target is 58
when the input is tensor([24, 43, 58]), the target is 5
when the input is tensor([24, 43, 58,  5]), the target is 57
when the input is tensor([24, 43, 58,  5, 57]), the target is 1
when the input is tensor([24, 43, 58,  5, 57,  1]), the target is 46
when the input is tensor([24, 43, 58,  5, 57,  1, 46]), the target is 43
when the input is tensor([24, 43, 58,  5, 57,  1, 46, 43]), the target is 39
when the input is tensor([44]), the target is 53
when the input is tensor([44, 53]), the target is 56
when the input is tensor([44, 53, 56]), the target is 1
when the input is tensor([44, 53, 56,  1]), the target is 58
when the input is tensor([44, 53, 56,  1, 58]), the target is 46
when the input is tensor([44, 53, 56,  1, 58, 46]), the target is 39
when the input is tensor([44, 53, 56,  1, 58, 46, 39]), the target is 58
when the input is tensor([44, 53, 56,  1, 58, 46, 39, 58]), the target i

In [12]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are of shape (B, T)
        logits = self.token_embedding_table(idx) # (B, T, C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # reshaping to (B*T, C) as torch expects C to be next to mini-batch
            targets = targets.view(B*T) # reshaping to (B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        # generate(...) should take (B, T) and generate (B, T+1),
        # take that and generate (B, T+2) until max_new_tokens
        for _ in range(max_new_tokens):
            # get the preds
            logits, _ = self(idx)
            # focus only on the last time step because it is bigram
            logits = logits[:, -1, :] # (B, C)
            # apply softmax
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
            

m = BigramLanguageModel(vocab_size)
out, loss = m(xb, yb)
print(loss)
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [14]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
# training loop
batch_size = 32
for steps in range(10000):
    # get batch
    xb, yb = get_batch('train')
    # forward pass
    logits, loss = m(xb, yb)
    # zero grad
    optimizer.zero_grad(set_to_none=True)
    # backward pass
    loss.backward()
    # parameter update
    optimizer.step()

In [16]:
print("Training loss: ", loss.item())
print(decode(m.generate(idx, max_new_tokens=1000)[0].tolist()))

Training loss:  2.5727508068084717

Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN t l-t E:
I hisgothers je are!-e!
QLYotouciullle'z,
Thitertho s?
NDan'spererfo cist ripl chys er orlese;
Yo jehof h hecere ek? wferommot mowo soaf yoit, ince his, t, f at. fal whetrimy bupof tor atha Bu!
JOutho f cimimave.
NEDUSt cir selle p wie wede
Ro n apenor f'Y tover witys an sh d w t e w!
CEOntiretoaveE IINpe, theck. cung.
ORIsthies hacin benqurd bll, d a r w wistatsowor ath
Fivet bloll ang a-I theeancu,
LINCI'T:
Sarry t I Ane sze t
LCKI thit,
n.
Faure ds ppplirn!
meftou ow pring, avewist th;
TENTEMETCI

In [ ]:
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)

In [24]:
# naive approach
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = xprev.mean(0) # (C)
xbow.shape

torch.Size([4, 8, 2])

In [29]:
# mathematical trick for vectorized implementation
# tril is lower triangular matrix
wei = torch.tril(torch.ones(T, T)) # (T, T)
wei = wei / wei.sum(1, keepdim=True) # (T, T)
xbow2 = wei @ x # (T, T) @ (B, T, C) -> (B, T, T) @ (B, T, C) -> (B, T, C)
print(xbow2.shape)
torch.allclose(xbow, xbow2)

torch.Size([4, 8, 2])


True

In [34]:
# version 3 using softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
print(xbow3.shape)
torch.allclose(xbow, xbow3)

torch.Size([4, 8, 2])


True

In [43]:
# Self attention
# Each token/node emits the following:
    # query - what do I want?
    # key - what do I contain?
    # value - what can I communicate to you if you find me interesting?
# matrix multiplying key and query will give affinities
# a high value in the output indicates that key and query are
# aligned and you will get to learn more about that pair in the network
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)

# Single headed self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, C) @ (B, C, 16) -> (B, T, 16)
q = query(x) # (B, T, C) @ (B, C, 16) -> (B, T, 16)
v = value(x) # (B, T, C) @ (B, C, 16) -> (B, T, 16)
wei = q @ k.transpose(-2, -1) * head_size**-0.5 # (B, T, 16) @ (B, 16, T) -> (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
out = wei @ v # (B, T, T) @ (B, T, 16) -> (B, T, 16)
out.shape

torch.Size([4, 8, 16])

In [40]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5599, 0.4401, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3220, 0.2016, 0.4764, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1640, 0.0815, 0.2961, 0.4585, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2051, 0.3007, 0.1894, 0.1808, 0.1241, 0.0000, 0.0000, 0.0000],
        [0.0600, 0.1273, 0.0291, 0.0169, 0.0552, 0.7114, 0.0000, 0.0000],
        [0.1408, 0.1025, 0.1744, 0.2038, 0.1690, 0.0669, 0.1426, 0.0000],
        [0.0223, 0.1086, 0.0082, 0.0040, 0.0080, 0.7257, 0.0216, 0.1016]],
       grad_fn=<SelectBackward0>)

In [2]:
import torch
torch.manual_seed(1337)

In [20]:
def batchnorm(x, eps=1e-5):
    # x: [N, C, H, W]
    N, C, H, W = x.shape
    
    gamma = torch.ones((1, C, 1, 1))
    beta = torch.zeros((1, C, 1, 1))
    
    xmean = x.mean([0, 2, 3], keepdim=True)
    xvar = x.var([0, 2, 3], keepdim=True)
    xhat = (x - xmean) / (torch.sqrt(xvar + eps))
    return gamma * xhat + beta

bn = torch.nn.BatchNorm2d(64)

# x: [N, C, H, W]
x = torch.randn(32, 64, 128, 128)
my_out = batchnorm(x)
torch_out = bn(x)
torch.allclose(my_out, torch_out)

True

In [23]:
def layernorm(x, eps=1e-5):
    # x: [N, C, H, W]
    N, C, H, W = x.shape
    
    gamma = torch.ones((N, 1, 1, 1))
    beta = torch.zeros((N, 1, 1, 1))
    
    xmean = x.mean([1, 2, 3], keepdim=True)
    xvar = x.var([1, 2, 3], keepdim=True)
    xhat = (x - xmean) / (torch.sqrt(xvar + eps))
    return gamma * xhat + beta

ln = torch.nn.LayerNorm([64, 128, 128])

# x: [N, C, H, W]
x = torch.randn(32, 64, 128, 128)
my_out = layernorm(x)
torch_out = ln(x)
torch.allclose(my_out, torch_out)

True

In [30]:
def instancenorm(x, eps=1e-5):
    # x: [N, C, H, W]
    N, C, H, W = x.shape
    
    gamma = torch.ones((N, C, 1, 1))
    beta = torch.zeros((N, C, 1, 1))
    
    xmean = x.mean([2, 3], keepdim=True)
    xvar = x.var([2, 3], keepdim=True)
    xhat = (x - xmean) / (torch.sqrt(xvar + eps))
    return gamma * xhat + beta

in_ = torch.nn.InstanceNorm2d(64)

# x: [N, C, H, W]
x = torch.randn(32, 64, 128, 128)
my_out = instancenorm(x)
torch_out = in_(x)
torch.allclose(my_out, torch_out)

False

In [41]:
def groupnorm(x, G, eps=1e-5):
    # x: [N, C, H, W]
    N, C, H, W = x.shape
    
    gamma = torch.ones((N, C, 1, 1))
    beta = torch.zeros((N, C, 1, 1))
    print(gamma.shape)

    x = torch.reshape(x, [N, G, C // G, H, W])
    xmean = x.mean([2, 3, 4], keepdim=True)
    print(xmean.shape)
    xvar = x.var([2, 3, 4], keepdim=True)
    xhat = (x - xmean) / (torch.sqrt(xvar + eps))

    xhat = torch.reshape(xhat, [N, C, H, W])
    
    return gamma * xhat + beta

gn = torch.nn.GroupNorm(8, 64)

# x: [N, C, H, W]
x = torch.randn(32, 64, 128, 128)
my_out = groupnorm(x, 8)
torch_out = gn(x)
torch.allclose(my_out, torch_out)

torch.Size([32, 64, 1, 1])
torch.Size([32, 8, 1, 1, 1])


True

In [39]:
torch_out.shape, my_out.shape

(torch.Size([32, 64, 128, 128]), torch.Size([32, 8, 8, 128, 128]))